 # Sugar tutorial





The objective of Sugar library is to provide a set of handy tools to facilitate the setup of simulations of polyelectrolytes and peptides with the Molecular Dynamics software ESPResSo. In other words: adding sugar to your ESPResSo scripts makes your life sweater. In this tutorial, the different functionalities of sugar are explained with different practical examples.

## Introduction

Let us get started by importing sugar library and creating an instance of it. When sugar is inicialitated, the default system of reduced units is defined:

In [1]:
import sugar
sg=sugar.sugar_library()


Current set of reduced units:
0.355 nanometer = 1 reduced_length
4.1164049935e-21 joule = 1 reduced_energy
Temperature: 298.15 kelvin
1.602176634e-19 coulomb = 1 reduced_charge



All input variables will be given to ESPResSo using these reduced units, since it is a convinient choice for the simulation setup. Internally, Sugar uses Pint library to deal with unit transformations, which in turn should be  used by the user to define its own variables

In [2]:
Box_L = 22 * sg.units.nm
print('The side of the simulation box is ', Box_L, '=' ,Box_L.to('reduced_length'))

The side of the simulation box is  22 nanometer = 61.971830985915496 reduced_length


However, the user is free to change the default equivalence to SI units to another of its own convienice, e.g.

In [3]:
sg.set_reduced_units(unit_length=0.5*sg.units.nm,  unit_charge=2*sg.units.e, temperature=300 * sg.units.K)


Current set of reduced units:
0.49999999999999994 nanometer = 1 reduced_length
4.141947e-21 joule = 1 reduced_energy
Temperature: 300 kelvin
3.204353268e-19 coulomb = 1 reduced_charge



The active set of reduced units can be consulted using

In [4]:
sg.print_reduced_units()


Current set of reduced units:
0.49999999999999994 nanometer = 1 reduced_length
4.141947e-21 joule = 1 reduced_energy
Temperature: 300 kelvin
3.204353268e-19 coulomb = 1 reduced_charge



## Using Sugar to build particles into espresso

The basic building block on Sugar library are particle objects, whose attributes define its chemical identity.

In [5]:
cation = sg.particle()
cation.q = 1 # Charge of the particle
cation.type = 0 # Desired particle type to be loaded in espresso
cation.radius = 0.1 * sg.units.nm # Particle radius
cation.N = 5 # Desired number of particles
cation.name = 'Na' # label to the object

anion = sg.particle()
anion.q = - 1 
anion.type = 1 
anion.radius = 0.1 * sg.units.nm
anion.N = 5
anion.name = 'Cl'

This particle objects then can be used as input of several sugar functions. For instance, one can use sugar to create them into an espresso system

In [6]:
import espressomd

system=espressomd.System(box_l=[Box_L.to('reduced_length').magnitude]*3)

sg.create_particle(system=system, particle=cation)
sg.create_particle(system=system, particle=anion)

for particle in system.part[:]:

    print('particle id', particle.id, 'particle type', particle.type,  'particle charge', particle.q)

particle id 0 particle type 0 particle charge 1.0
particle id 1 particle type 0 particle charge 1.0
particle id 2 particle type 0 particle charge 1.0
particle id 3 particle type 0 particle charge 1.0
particle id 4 particle type 0 particle charge 1.0
particle id 5 particle type 1 particle charge -1.0
particle id 6 particle type 1 particle charge -1.0
particle id 7 particle type 1 particle charge -1.0
particle id 8 particle type 1 particle charge -1.0
particle id 9 particle type 1 particle charge -1.0


All the information stored in the particle object (or any other sugar molecule object) can be easilly consulted 

In [7]:
sg.write_parameters(mol=cation)

Particle Na
('N', 5)
('acidity', None)
('ids', [0, 1, 2, 3, 4])
('model', None)
('name', 'Na')
('pKa', None)
('q', 1)
('radius', <Quantity(0.1, 'nanometer')>)
('state', None)
('type', 0)


In the case of salt ions, these particle objects can be also used to create a given added salt concentration to your simulation. Please, note that in the current version of sugar only 1:1 salt can be created with these function.

In [8]:
c_salt=1e-3*sg.units.mol/sg.units.L

c_salt_calculated=sg.create_added_salt(system=system, cation=cation, anion=anion, c_salt=c_salt)


 Added an added salt concentration of  0.0009488663004518591 mole / liter given by  17 cations/anions


## Using sugar to build macromolecules in espresso

In [9]:
sequence='A'*10+'H'*10
custom_dict = {'A': {'type': 2,
                'q': +1,
                'radius': 0.2*sg.units.nm,
                            },
               'H': {'type': 3,
                'q': -1,
                'radius': 0.2*sg.units.nm,
                            }
              }



customp=sg.create_custom_model(custom_particles=custom_dict, principal_chain={"default": "sequence"})

test_molecule=sg.molecule(pKa_set='hass',sequence=sequence, param_custom=customp)

sg.write_parameters(test_molecule)

sg.param.default.default_bond.k=0.4*sg.units.N/sg.units.m

test_molecule.N=1

sg.create_molecule(system=system, mol=test_molecule)


molecule parameters
('N', None)
('Nm', 20)
('ids', [])
('model', <sugar.sugar_library.parameters.custom_parameters object at 0x7f6904285160>)
('residues', [[<sugar.sugar_library.residue object at 0x7f691c2e9f10>], [<sugar.sugar_library.residue object at 0x7f691c2e9a00>], [<sugar.sugar_library.residue object at 0x7f691c2e9eb0>], [<sugar.sugar_library.residue object at 0x7f691c2e9f70>], [<sugar.sugar_library.residue object at 0x7f691c2e9700>], [<sugar.sugar_library.residue object at 0x7f691c2e3b80>], [<sugar.sugar_library.residue object at 0x7f691c2e31c0>], [<sugar.sugar_library.residue object at 0x7f691c2e3af0>], [<sugar.sugar_library.residue object at 0x7f691c2e34c0>], [<sugar.sugar_library.residue object at 0x7f68c14024f0>], [<sugar.sugar_library.residue object at 0x7f68c14020d0>], [<sugar.sugar_library.residue object at 0x7f68c14022e0>], [<sugar.sugar_library.residue object at 0x7f68c1402430>], [<sugar.sugar_library.residue object at 0x7f6904296070>], [<sugar.sugar_library.residue ob

Practically, to create a peptide object one only needs to provide the aminoacid sequence to the library. The prefered format is a single string using the one-letter aminoacid code of the peptide. The amino and carboxyl ends are denoted as 'c' and 'n' respectively and must be provided in lower case. 

Let us ilustrate this concept with the example of a peptide. A peptide is a macromolecule which is composed by several aminoacids which, in turn, have several chemical groups. In the same way, in this library a peptide object contains several aminoacid objects. In turn, each aminoacid object contains bead objects, whose number depends on the desired model.

The library internally works with the one letter aminoacid code. However, the user can also provide the sequence in the three-letter format, separated by hyphens

or a mix between both nomenclatures, including lower/upper case  missprints. Note that one needs to be carefull with missprints in "C" and "N" letters since the lower/upper case have different meaning (e.g. "c" = COOH and "C" = CYS)

The library includes a sequence parser that reads the input sequence and automatically transforms it to the one-letter nomenclature. As a result, examples 1-3 yield to the creation of the same protein object.

By default, the library assumes that the peptide is modelled as a linear chain with one bead per aminoacid. However, the user can also choose to model the peptide as a chain with side chains, i.e. with two beads per aminoacid.

Regarding the pKa-values of the aminoacids, the library contains different stablished sets of values that can be loaded using the following keywords: "hass" (Ref. 1), "platzer" (Ref. 2), "crchandbook" (Ref. 3) or "nozaki" (Ref. 4). Values for the phosphorilated aminoacids J (pSER) U (pTHR) and Z (pTYR) are always taken from Ref. 5. By default, the library asumes the pKa-values provided in Ref. 1 (Hass et al.). However, the user can choose another set of pKa-values:

Alternatively, the user can provide custom pKa-values for all or some of the aminoacids. The custom pKa-values must be provided in a dictionary:

 The pKa-values not specified by the user are assumed to be equal to those provided in Ref. 1 (Hass et al.).
 
 Once the user creates the peptide object with the desired properties, the library contains several useful modules that help the user to set-up a constant pH simulation of such peptide with Espresso. For instance, let us consider the case that you want to simulate two peptide chains of sequence "cGHACCFVn" using the two bead model and the default pKa-values. 


The library can be used to directly create the peptide into the espresso system.  The user if the initial state of the titrable aminoacid is "neutral" or "charged", by default the neutral case is considered.


The ids  of all the particles created in the system are stored in the peptide object and its sub-objects. Ids belonging to different peptide chains are stored in separated lists.

The user can also create counter ions particles of the peptide chains using the module provided in this library

which returns particle objects containing all the information of the counter-ions generated

The library can be used to quickly set-up all the acid/base reactions present in the peptide

The library can also be used to calculate the total number of acidic/basic groups in peptide chains,

and to set-up espresso to track in which state are the ionizable groups during the simulation

At this stage, the system is ready to run a simulation with Espresso. Here, we ilustrate its use for a set of simulation done for pH values ranging from 2 to 12

The results of the simulation should be compared with the Henderson-Hasselbach analytical prediction for an ideal system. The library also provides a module that computes the ideal peptide charge predicted by  Henderson-Hasselbach theory for a given set of pH values:

References

[1] Hass MA, Mulder FAA. Contemporary NMR Studies of Protein Electrostatics. Annu Rev Biophys. 2015;44:53-75.

[2] Platzer G, Okon M, McIntosh LP. 2014. pH-dependent random coil 1 H, 13 C, and 15 N chemical shifts of the ionizable amino acids: a guide for protein pK a measurements. J. Biomol. NMR 60:109–29

[3] Handbook of Chemistry and Physics, 72nd Edition, CRC Press, Boca Raton, FL, 1991.

[4] Y. Nozaki and C. Tanford, Methods Enzymol., 1967, 11, 715–734.

[5] Bienkiewicz & K.J. Lumb, J Biomol NMR 15: 203-206 (1999).

